In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

In [2]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [3]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True    

In [4]:
# 종 분류 모델 가져오기
from tensorflow.compat.v2.keras.models import model_from_json

# json 파일 열기-v2
json_file = open('CNN_dog_breed_model_v2.json','r')
loaded_model_json = json_file.read()
json_file.close()

# json 파일로부터 model 로드하기
loaded_model = model_from_json(loaded_model_json)

# 로드한 model에 weight 로드하기
loaded_model.load_weights('CNN_dog_breed_model_v2.h5')

In [5]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input as preprocess_input_vgg19
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50

def extract_VGG19(file_paths):
    tensors = paths_to_tensor(file_paths).astype('float32')
    preprocessed_input = preprocess_input_vgg19(tensors)
    return VGG19(weights='imagenet', include_top=False).predict(preprocessed_input, batch_size=32)

def extract_Resnet50(file_paths):
    tensors = paths_to_tensor(file_paths).astype('float32')
    preprocessed_input = preprocess_input_resnet50(tensors)
    return ResNet50(weights='imagenet', include_top=False).predict(preprocessed_input, batch_size=32)

In [6]:
from keras.callbacks import ModelCheckpoint  

EPOCHS = 10
loaded_model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='saved_models/bestmodel.hdf5', 
                               verbose=1, save_best_only=True)

In [9]:
predic = loaded_model.predict([extract_VGG19(['dogs/valid/001.Affenpinscher/Affenpinscher_00041.jpg']),extract_Resnet50(['dogs/valid/001.Affenpinscher/Affenpinscher_00041.jpg'])])
print(np.argmax(predic))
print(np.argmax(loaded_model.predict([extract_VGG19(['dogs/valid/019.Bedlington_terrier/Bedlington_terrier_01357.jpg']),extract_Resnet50(['dogs/valid/019.Bedlington_terrier/Bedlington_terrier_01357.jpg'])])))
# ??? 왜 서버에선 잘되지?

100%|██████████| 1/1 [00:00<00:00, 340.28it/s]


100%|██████████| 1/1 [00:00<00:00, 271.65it/s]


100%|██████████| 1/1 [00:00<00:00, 21.67it/s]

0


100%|██████████| 1/1 [00:00<00:00, 154.69it/s]


18
